# SLAM Unit E
In this unit, you will learn about the particle filter.

## Why we would want another representation for the belief

In [1]:
# If you don't see a video below, run this cell.
# YouTube = True  # Uncomment to get YouTube videos instead of TIB AV.
from IPython.display import IFrame
IFrame("https://www.youtube.com/embed/ME7kUbLYMno" if "YouTube" in globals() else "//av.tib.eu/player/49017",
       width=560, height=315)

In [2]:
# Which alternative is the correct solution, "A", "B", or "C"?
bayes_can_handle = "C"
# YOUR CODE HERE
#raise NotImplementedError()

In [3]:
from hashlib import shake_128
def public_string_test(the_answer_string, reference):
    m = shake_128()
    m.update(the_answer_string.encode())
    return m.hexdigest(4) == reference
assert public_string_test("bayes_says="+bayes_can_handle.strip().lower(), 'ebd1fdee'), "Oh no, your answer is wrong!"

In [4]:
# If you don't see a video below, run this cell.
from IPython.display import IFrame
IFrame("https://www.youtube.com/embed/N6-1aGWE9IM" if "YouTube" in globals() else "//av.tib.eu/player/49018",
       width=560, height=315)

In [5]:
# Is it possible to model the belief using a very wide normal distribution, "Yes" or "No"?
wide_normal_possible = "Yes"
# YOUR CODE HERE
#raise NotImplementedError()

In [6]:
from hashlib import shake_128
def public_string_test(the_answer_string, reference):
    m = shake_128()
    m.update(the_answer_string.encode())
    return m.hexdigest(4) == reference
assert public_string_test("it is="+wide_normal_possible.strip().lower(), 'b41dfcdc'), "Oh no, your answer is wrong!"

In [7]:
# If you don't see a video below, run this cell.
from IPython.display import IFrame
IFrame("https://www.youtube.com/embed/dgvi7eLleOo" if "YouTube" in globals() else "//av.tib.eu/player/49019",
       width=560, height=315)

In [8]:
# Can we use the "large variance initialization" trick with the Kalman filter localization
# approach we developed in the previous unit? Answer with "A", "B", or "C".
wide_normal_possible_too = "C"
# YOUR CODE HERE
#raise NotImplementedError()

In [9]:
from hashlib import shake_128
def public_string_test(the_answer_string, reference):
    m = shake_128()
    m.update(the_answer_string.encode())
    return m.hexdigest(4) == reference
assert public_string_test("maybe="+wide_normal_possible_too.strip().lower(), 'f84fab62'), "Oh no, your answer is wrong!"

## Introducing the particle filter: the prediction step

In [10]:
# If you don't see a video below, run this cell.
from IPython.display import IFrame
IFrame("https://www.youtube.com/embed/9jhnye7I2pU" if "YouTube" in globals() else "//av.tib.eu/player/49020",
       width=560, height=315)

### Programming assignment: program the prediction step of the particle filter (15 Points).

In [11]:
# The particle filter, prediciton only.
#
# slam_08_a_particle_prediciton.
from lego_robot import *
from math import sin, cos, pi, sqrt
from random import gauss as carl_friedrich
import random
# Use this class as an entry to random functions.
class Random:
    @staticmethod
    def gauss(mu, sigma):
        return carl_friedrich(mu, sigma)

class ParticleFilter_1:
    def __init__(self, initial_particles,
                 robot_width, scanner_displacement,
                 control_motion_factor, control_turn_factor):
        # The random generator to use.
        self.random = Random

        # The particles.
        self.particles = initial_particles

        # Some constants.
        self.robot_width = robot_width
        self.scanner_displacement = scanner_displacement
        self.control_motion_factor = control_motion_factor
        self.control_turn_factor = control_turn_factor

    # State transition. This is exactly the same method as in the Kalman filter.
    @staticmethod
    def g(state, control, w):
        x, y, theta = state
        l, r = control
        if r != l:
            alpha = (r - l) / w
            rad = l/alpha
            g1 = x + (rad + w/2.)*(sin(theta+alpha) - sin(theta))
            g2 = y + (rad + w/2.)*(-cos(theta+alpha) + cos(theta))
            g3 = (theta + alpha + pi) % (2*pi) - pi
        else:
            g1 = x + l * cos(theta)
            g2 = y + l * sin(theta)
            g3 = theta

        return (g1, g2, g3)

    def predict(self, control):
        """The prediction step of the particle filter."""
        left, right = control

        # Compute left and right variance.
        # alpha_1 is self.control_motion_factor.
        # alpha_2 is self.control_turn_factor.
        # Then, do a loop over all self.particles and construct a new
        # list of particles.
        # In the end, assign the new list of particles to self.particles.
        # NOTE: For sampling, use self.random.gauss(mu, sigma).
        # (Note sigma in this call is the standard deviation,
        # not the variance.)
        # DO NOT import and use other sampling modules, as this will break
        # the tests.

        # YOUR CODE HERE
        

        sigma_l = (self.control_motion_factor*left)**2 + (self.control_turn_factor*(left-right))**2
        sigma_r = (self.control_motion_factor*right)**2 + (self.control_turn_factor*(left-right))**2
        particles_predict = []
        
        for i in self.particles:
            sample_l = self.random.gauss(left, sqrt(sigma_l))
            sample_r = self.random.gauss(right, sqrt(sigma_r))
            control_sample = [sample_l,sample_r]
            particles_predict.append(self.g(i, control_sample, self.robot_width))
        
        self.particles = particles_predict
        
        
        
        #raise NotImplementedError()

    def print_particles(self, file_desc):
        """Prints particles to given file_desc output."""
        if not self.particles:
            return
        print("PA", end=' ', file=file_desc)
        for p in self.particles:
            print("%.0f %.0f %.3f" % p, end=' ', file=file_desc)
        print(file=file_desc)


if __name__ == '__main__':
    # Robot constants.
    scanner_displacement = 30.0
    ticks_to_mm = 0.349
    robot_width = 155.0

    # Filter constants.
    control_motion_factor = 0.35  # Error in motor control.
    control_turn_factor = 0.6  # Additional error due to slip when turning.

    # Generate initial particles. Each particle is (x, y, theta).
    number_of_particles = 300
    measured_state = (1850.0, 1897.0, 213.0 / 180.0 * pi)
    standard_deviations = (100.0, 100.0, 10.0 / 180.0 * pi)
    initial_particles = []
    for i in range(number_of_particles):
        initial_particles.append(tuple([
            Random.gauss(measured_state[j], standard_deviations[j])
            for j in range(3)]))

    # Setup filter.
    pf = ParticleFilter_1(initial_particles,
                          robot_width, scanner_displacement,
                          control_motion_factor, control_turn_factor)

    # Read data.
    logfile = LegoLogfile()
    logfile.read("robot4_motors.txt")

    # Loop over all motor tick records.
    # This is the particle filter loop, with prediction and correction.
    f = open("particle_filter_predicted.txt", "w")
    for i in range(len(logfile.motor_ticks)):
        # Prediction.
        control = [x * ticks_to_mm for x in logfile.motor_ticks[i]]
        pf.predict(control)

        # Output particles.
        pf.print_particles(f)

    f.close()

### First, let's have a look at the results.

In [12]:
# Execute this to run the interactive viewer.
import ipy_logfile_viewer as lfv
v = lfv.IPYLogfileViewer(files=["particle_filter_predicted.txt", "robot4_reference.txt"])

### Here is the test for the prediction.

In [13]:
# Particle filter predicion test.
from random import uniform
class Random_hook:
    def __init__(self):
        self.log = {}
    def gauss(self, mu, sigma):
        key = uniform(0,1)
        self.log[key] = (mu, sigma)
        return key

def public_test(the_pf_class):
    class TestPF(the_pf_class):
        def __init__(self, ip, rw, sd, cmf, ctf):
            super().__init__(ip, rw, sd, cmf, ctf)
            self.random = Random_hook()
            self.g = self.g_hook
            self.log = {}

        def g_hook(self, state, control, w):
            key = uniform(0,1)
            self.log[key] = (state, control)
            return (key, key, key)

    # Constants.
    scanner_displacement = uniform(10, 40)
    robot_width = uniform(100, 200)
    control_motion_factor = uniform(0.2, 0.5)
    control_turn_factor = uniform(0.4, 0.8)

    # Draw initial particles and run one prediction step.
    particle_number = 10
    init_part = [(uniform(-10,10), uniform(-10,10), uniform(-3.14, 3.14))\
                 for _ in range(particle_number)]
    pf = TestPF(init_part[:],
                robot_width, scanner_displacement,
                control_motion_factor, control_turn_factor)
    l, r = uniform(-1,1), uniform(2,3)
    pf.predict((l, r))
    
    # Tests. Checks chain from end to beginning.
    # Must have correct number of output particles.
    if len(pf.particles) != particle_number:
        print("Generated wrong number of particles.")
        return False
    # All paticles must be distinct.
    if len(set(pf.particles)) != particle_number:
        print("Generated identical particles.")
        return False
    # For each particle, get state and control.
    g_in = [pf.log[s[0]] for s in pf.particles]
    # The particle states must be exactly the initial particles.
    if set(init_part) != set(s for s,c in g_in):
        print("Called g() with wrong particle states.")
        return False
    # Now follow (l,r). All must have been different random calls.
    ctrl = [c for s,c in g_in]
    ctrl_flat = set(c for pair in ctrl for c in pair)
    if len(ctrl_flat) != 2*particle_number:
        print("Seems you did not randomize each particle.")
        return False
    # All used random calls must have the same (l, sigma_l) or (r, sigma_r).
    lefts  = set(pf.random.log[c[0]] for c in ctrl)
    rights = set(pf.random.log[c[1]] for c in ctrl)
    if len(lefts) != 1 or len(rights) != 1:
        print("Called self.random.gauss with varying parameter values?")
        return False
    # Finally, check if stddev are correct.
    lval, sl, rval, sr = lefts.pop() + rights.pop()
    l2, r2, sl2, sr2 = l*l, r*r, sl*sl, sr*sr
    a12, a22 = control_motion_factor**2, control_turn_factor**2
    if lval != l or rval != r or\
       abs(sl2-sr2 - a12*(l2-r2)) > 1e-5 or\
       abs(sl2+sr2 - (a12+2*a22)*(l2+r2)+4*a22*l*r) > 1e-5:
        print("Called self.random.gauss with wrong parameter values.")
        return False
    return True
assert public_test(ParticleFilter_1)

## Introducing the particle filter correction step

In [14]:
# If you don't see a video below, run this cell.
from IPython.display import IFrame
IFrame("https://www.youtube.com/embed/kZhOJgooMxI" if "YouTube" in globals() else "//av.tib.eu/player/49021",
       width=560, height=315)

### Programming assignment: program the correction step, first part: `probability_of_measurement` (10 Points).

In [15]:
# The particle filter, prediciton and correction.
#
# slam_08_b_particle_correction.
from math import pi
from scipy.stats import norm as normal_dist

class ParticleFilter_2(ParticleFilter_1):
    def __init__(self, initial_particles,
                 robot_width, scanner_displacement,
                 control_motion_factor, control_turn_factor,
                 measurement_distance_stddev, measurement_angle_stddev):
        super().__init__(initial_particles, robot_width, scanner_displacement,
                         control_motion_factor, control_turn_factor)
        self.measurement_distance_stddev = measurement_distance_stddev
        self.measurement_angle_stddev = measurement_angle_stddev

    def probability_of_measurement(self, measurement, predicted_measurement):
        """Given a measurement and a predicted measurement, computes
           probability."""
        # Compute differences to real measurements.

        # --->>> Compute difference in distance and bearing angle.
        # Important: make sure the angle difference works correctly and does
        # not return values offset by 2 pi or - 2 pi.
        # You may use the following Gaussian PDF function:
        #   normal_dist.pdf(x, mu, sigma).
        # Note that the two parameters sigma_d and sigma_alpha discussed
        # in the lecture are self.measurement_distance_stddev and
        # self.measurement_angle_stddev.
        
        # YOUR CODE HERE
        
        dist,alpha = measurement
        dist_pre, alpha_pre = predicted_measurement
        
        dist_diff = dist - dist_pre
        alpha_diff = (alpha - alpha_pre + pi) % (2*pi) - pi
            
        alpha_prob = normal_dist.pdf(alpha_diff, 0, self.measurement_angle_stddev)
        dist_prob = normal_dist.pdf(dist_diff, 0, self.measurement_distance_stddev)
            
        return alpha_prob * dist_prob
        
        #raise NotImplementedError()

In [16]:
# First test: function probability_of_measurement().
from random import uniform
from scipy.stats import norm as normal_dist
from math import pi

def test_probability_of_measurement(the_pf_class):

    d_std, a_std = 150.0, 20.0 / 180.0 * pi

    # Setup filter.
    pf = the_pf_class([], 140, 40, 0.4, 0.7, d_std, a_std)

    # Test this for the full circle.
    steps = 50
    bad_steps = 0
    for i in range(0,steps+1):
        d1, dd = uniform(d_std, 1000), uniform(-d_std,d_std)
        a1, ad = (i*2*pi/steps-pi), uniform(-a_std,a_std)
        p_ref = normal_dist.pdf(dd, 0, d_std) * normal_dist.pdf(2*ad, 0, a_std)
        p = pf.probability_of_measurement(
            (d1, (a1-ad+pi)%(2*pi)-pi), (d1+dd, (a1+ad+pi)%(2*pi)-pi))
        if abs(p-p_ref) > 1e-10:
            bad_steps += 1
    if bad_steps > steps/2:
        print("Wrong implementation in probability_of_measurement().")
        return False
    elif bad_steps > 0:
        print("Check handling of angle differences in "\
              "probability_of_measurement().")
        return False
    return True
assert(test_probability_of_measurement(ParticleFilter_2))

### Programming assignment: program the correction step, second part: `compute_weights` (10 Points).

In [17]:
# The particle filter, prediciton and correction.
#
# slam_08_b_particle_correction.
from slam_e_library import assign_cylinders
from math import atan2, pi

class ParticleFilter_3(ParticleFilter_2):
    # Measurement. This is exactly the same method as in the Kalman filter.
    @staticmethod
    def h(state, landmark, scanner_displacement):
        """Takes a (x, y, theta) state and a (x, y) landmark, and returns the
           corresponding (range, bearing)."""
        dx = landmark[0] - (state[0] + scanner_displacement * cos(state[2]))
        dy = landmark[1] - (state[1] + scanner_displacement * sin(state[2]))
        r = sqrt(dx * dx + dy * dy)
        alpha = (atan2(dy, dx) - state[2] + pi) % (2*pi) - pi
        return (r, alpha)

    def compute_weights(self, cylinders, landmarks):
        """Computes one weight for each particle, return list of weights."""
        weights = []
        for p in self.particles:
            # Get list of tuples:
            # [ ((range_0, bearing_0), (landmark_x, landmark_y)), ... ]
            assignment = assign_cylinders(cylinders, p,
                self.scanner_displacement, landmarks)
            # --->>> Insert code to compute weight for particle p here.
            # This will require a loop over all (measurement, landmark)
            # in assignment. Append weight to the list of weights using
            # weights.append(...).
            # YOUR CODE HERE
            
            weight = 1.0
            
            for pair in assignment:
                measurement, landmark = pair
                predicted_measurement = self.h(p, landmark, self.scanner_displacement)
                weight *= self.probability_of_measurement(measurement, predicted_measurement)
            
            weights.append(weight)  

            #raise NotImplementedError()
            
        return weights

In [18]:
# Second test: function compute_weights().
from random import uniform as uf
from random import randrange
from math import pi, sin, cos, isclose
from operator import mul
from functools import reduce
from slam_e_library import assign_cylinders as ass
# Given state and landmark, get (x,y) in scanner coord system.
def euc(state, xy, sd):
    x,y,ct,st = state[0], state[1], cos(state[2]), sin(state[2])
    xv, yv = xy[0]-x-sd*ct, xy[1]-y-sd*st
    return (xv*ct+yv*st, yv*ct-xv*st)
def test_compute_weights(the_pf_class):
    num_experiments = 10
    num_particles = 5
    num_landmarks = 3
    for _ in range(num_experiments):
        part = [(uf(0,200),uf(0,200),uf(-pi,pi)) for _ in range(num_particles)]
        lm = [(uf(0,200),uf(0,200)) for _ in range(num_landmarks)]
        pf = the_pf_class(part[:], 140, 40, 0.3, 0.5, 180, 0.25)
        p0 = part[randrange(num_particles)]
        obs = [pf.h(p0,l,40)+euc(p0,l,40) for l in lm]
        w = pf.compute_weights(obs,lm)
        if len(w) != num_particles:
            print("Error: compute_weights must produce one weight"\
                  "per particle.")
            return False
        if not all(isclose(a,b) for a,b in zip(w,(reduce(mul,(
           pf.probability_of_measurement(m,pf.h(p,l,40))
           for m,l in ass(obs,p,40,lm))) for p in part))):
            print("Error: compte_weights() produces wrong weights.")
            return False
    return True
assert(test_compute_weights(ParticleFilter_3))

### Programming assignment: program the correction step, third part: `resample` (10 Points).

**Please note this hint:** In the video, `offset += random.uniform(0, 2*max_weight)` is proposed to move the pointer forward. While this works in general, it introduces a small error. Instead, use `offset += random.uniform(0, sum_weight)`, where `sum_weight` is the sum of all weights. (Note that Python has a built-in function `sum` which can be used to compute the sum of an iterable (e.g., a list).) If you are interested in this topic, you will find more under the keyword *low variance sampling*, e.g. in the Probabilistic Robotics Book of Thrun, Burgard and Fox (Table 4.4 in the book). If you like, you can also implement the program listed there.

In [19]:
# The particle filter, prediciton and correction.
#
# slam_08_b_particle_correction.
import random

class ParticleFilter_4(ParticleFilter_3):

    def resample(self, weights):
        """Return a list of particles which have been resampled, proportional
           to the given weights."""
        # You may implement the 'resampling wheel' algorithm
        # described in the lecture.
        # HINT: In the video, offset += random.uniform(0, 2*max_weight) is proposed.
        # Instead, use offset += random.uniform(0, sum_weight), where sum_weight is
        # the sum of all weights.
        # YOUR CODE HERE
        
        new_particles = []
        max_weight = sum(weights) # max of weights
        Length_weights = len(weights) # length of weights
        offset = 0.0
        index = random.randint(0,Length_weights-1)
        
        for i in range(Length_weights):
            offset += random.uniform(0, 2*max_weight)
            while (offset > weights[index]):
                offset -= weights[index]
                index = (index+1) % Length_weights
            new_particles.append(self.particles[index])
            
        #raise NotImplementedError()
        
        return new_particles

In [20]:
# Third test: function resample().
import numpy as np
def test_resample(the_pf_class):
    # Setup filter.
    num_particles = 5
    experiment_loops = 4
    outer_loops = 50
    inner_loops = 10000
    pf = the_pf_class([(i,i,i) for i in range(num_particles)],
                      140, 40, 0.3, 0.5, 180, 0.25)
    # Experiment is OK if it succeeds once.
    for e in range(experiment_loops):
        # Make up some weights. Force them to be a bit different by
        # adding arange, shuffle to prevent always increasing values.
        weights = np.random.uniform(size=num_particles)+np.arange(num_particles)
        np.random.shuffle(weights)
        weights /= weights.sum()
        histo = np.zeros(num_particles)  # Histogram to collect choices.
        for o in range(1, outer_loops):
            for i in range(inner_loops):
                np.add.at(histo, np.array(pf.resample(weights))[:,0], 1)
            rel_freq = histo / (num_particles*inner_loops*o)
            # You may uncomment this to check how the error develops.
            #print(np.max(np.abs(rel_freq-weights)))
            if np.max(np.abs(rel_freq-weights)) < 1e-3:
                return True
    print("It seems your draws do not implement the correct probabilities.")
    return False
assert(test_resample(ParticleFilter_4))

### Not, we put everything together!
Running this will generate the file `particle_filter_corrected.txt` which we will look at in the next but one cell.

In [21]:
# Execute this to generate particle_filter_corrected.txt.
# The particle filter, prediciton and correction.
#
# slam_08_b_particle_correction.
from lego_robot import *
from slam_e_library import get_cylinders_from_scan
from math import pi
import random

class ParticleFilter_5(ParticleFilter_4):
    
    # Note there is no need to copy & paste your functions above to here, since
    # they are inherited.

    def correct(self, cylinders, landmarks):
        """The correction step of the particle filter."""
        # First compute all weights.
        weights = self.compute_weights(cylinders, landmarks)
        # Then resample, based on the weight array.
        self.particles = self.resample(weights)

    def print_particles(self, file_desc):
        """Prints particles to given file_desc output."""
        if not self.particles:
            return
        print("PA", end=' ', file=file_desc)
        for p in self.particles:
            print("%.0f %.0f %.3f" % p, end=' ', file=file_desc)
        print(file=file_desc)

if __name__ == '__main__':
    # Robot constants.
    scanner_displacement = 30.0
    ticks_to_mm = 0.349
    robot_width = 155.0

    # Cylinder extraction and matching constants.
    minimum_valid_distance = 20.0
    depth_jump = 100.0
    cylinder_offset = 90.0

    # Filter constants.
    control_motion_factor = 0.35  # Error in motor control.
    control_turn_factor = 0.6  # Additional error due to slip when turning.
    measurement_distance_stddev = 200.0  # Distance measurement error of cylinders.
    measurement_angle_stddev = 15.0 / 180.0 * pi  # Angle measurement error.

    # Generate initial particles. Each particle is (x, y, theta).
    number_of_particles = 50
    measured_state = (1850.0, 1897.0, 213.0 / 180.0 * pi)
    standard_deviations = (100.0, 100.0, 10.0 / 180.0 * pi)
    initial_particles = []
    for i in range(number_of_particles):
        initial_particles.append(tuple([
            random.gauss(measured_state[j], standard_deviations[j])
            for j in range(3)]))

    # Setup filter.
    pf = ParticleFilter_5(initial_particles,
                          robot_width, scanner_displacement,
                          control_motion_factor, control_turn_factor,
                          measurement_distance_stddev,
                          measurement_angle_stddev)

    # Read data.
    logfile = LegoLogfile()
    logfile.read("robot4_motors.txt")
    logfile.read("robot4_scan.txt")
    logfile.read("robot_arena_landmarks.txt")
    reference_cylinders = [l[1:3] for l in logfile.landmarks]

    # Loop over all motor tick records.
    # This is the particle filter loop, with prediction and correction.
    f = open("particle_filter_corrected.txt", "w")
    for i in range(len(logfile.motor_ticks)):
        # Prediction.
        control = [x * ticks_to_mm for x in logfile.motor_ticks[i]]
        pf.predict(control)

        # Correction.
        cylinders = get_cylinders_from_scan(logfile.scan_data[i], depth_jump,
            minimum_valid_distance, cylinder_offset)
        pf.correct(cylinders, reference_cylinders)

        # Output particles.
        pf.print_particles(f)

    f.close()

### Let's have a look in the logfile viewer.

In [22]:
# Execute this to run the interactive viewer.
import ipy_logfile_viewer as lfv
v = lfv.IPYLogfileViewer(files=["particle_filter_corrected.txt", "robot4_reference.txt"])

## Density estimation
Now, we have a lot of particles, but where is the robot?

In [23]:
# If you don't see a video below, run this cell.
from IPython.display import IFrame
IFrame("https://www.youtube.com/embed/x-bHZMo1a28" if "YouTube" in globals() else "//av.tib.eu/player/49022",
       width=560, height=315)

### Programming assignment: compute the mean state (10 Points).

In [24]:
class ParticleFilter_6(ParticleFilter_5):

    def get_mean(self):
        """Compute mean position and heading from all particles."""
        # Return a tuple: (mean_x, mean_y, mean_heading).

        # YOUR CODE HERE
        
        sum_x = 0.0
        sum_y = 0.0
        sum_vx = 0.0
        sum_vy = 0.0 
        Length_particles = len(self.particles)
        
        for p in self.particles:
            sum_x += p[0]
            sum_y += p[1]
            sum_vx += cos(p[2])
            sum_vy += sin(p[2])
            
        mean_x = sum_x/Length_particles
        mean_y = sum_y/Length_particles
        mean_heading = atan2(sum_vy, sum_vx)
        
        return (mean_x, mean_y, mean_heading) 
        #raise NotImplementedError()


In [25]:
# Test the get_mean function.
from random import randrange
import numpy as np
def test_mean(the_pf_class):
    n = 20 + 2*randrange(10)  # Random number of particles.
    xyt = np.random.uniform(-1,1,size=(n,3))
    xyt[n//2:n,:] = -xyt[0:n//2,:]
    xytc = np.random.uniform(-5,5,3)  # Random center.
    xytc[2] = np.random.uniform(-0.1, 0.1)  # Heading.
    xyts = xyt + xytc  # Shift.
    # Run through get_mean and test.
    pf = the_pf_class([tuple(p) for p in xyts], 140, 40, 0.3, 0.5, 180, 0.25)
    c = pf.get_mean()
    if (abs(c-xytc) > 1e-10).any():
        print("Your get_mean() produces the wrong result.")
        return False
    # Do the same once more for dangerous heading angles.
    xytc = np.random.uniform(-5,5,3)
    xytc[2] = np.random.uniform(-0.1, 0.1) - np.pi
    xyts = xyt + xytc
    xyts[:,2] = (xyts[:,2]+np.pi)%(2*np.pi)-np.pi  # Normalize -pi..pi range.
    pf = the_pf_class([tuple(p) for p in xyts], 140, 40, 0.3, 0.5, 180, 0.25)
    c = np.array(pf.get_mean())
    # Normalize angle for reference and result.
    xytc[2] = (xytc[2]+np.pi)%(2*np.pi)-np.pi
    c[2] = (c[2]+np.pi)%(2*np.pi)-np.pi
    if (abs(c-xytc) > 1e-10).any():
        print("Wrong mean - check your computation of the heading.")
        return False
    return True
assert(test_mean(ParticleFilter_6))

### Run the following cell to produce the output.

In [26]:
# The particle filter, prediciton and correction.
# In addition to the filtered particles, the density is estimated. In this
# simple case, the mean position and heading is computed from all particles.
#
# slam_08_c_density_estimation.
from lego_robot import *
from slam_e_library import get_cylinders_from_scan
from math import sin, cos, pi

if __name__ == '__main__':
    # Robot constants.
    scanner_displacement = 30.0
    ticks_to_mm = 0.349
    robot_width = 155.0

    # Cylinder extraction and matching constants.
    minimum_valid_distance = 20.0
    depth_jump = 100.0
    cylinder_offset = 90.0

    # Filter constants.
    control_motion_factor = 0.35  # Error in motor control.
    control_turn_factor = 0.6  # Additional error due to slip when turning.
    measurement_distance_stddev = 200.0  # Distance measurement error of cylinders.
    measurement_angle_stddev = 15.0 / 180.0 * pi  # Angle measurement error.

    # Generate initial particles. Each particle is (x, y, theta).
    number_of_particles = 50
    measured_state = (1850.0, 1897.0, 213.0 / 180.0 * pi)
    standard_deviations = (100.0, 100.0, 10.0 / 180.0 * pi)
    initial_particles = []
    for i in range(number_of_particles):
        initial_particles.append(tuple([
            random.gauss(measured_state[j], standard_deviations[j])
            for j in range(3)]))

    # Setup filter.
    pf = ParticleFilter_6(initial_particles,
                          robot_width, scanner_displacement,
                          control_motion_factor, control_turn_factor,
                          measurement_distance_stddev,
                          measurement_angle_stddev)

    # Read data.
    logfile = LegoLogfile()
    logfile.read("robot4_motors.txt")
    logfile.read("robot4_scan.txt")
    logfile.read("robot_arena_landmarks.txt")
    reference_cylinders = [l[1:3] for l in logfile.landmarks]

    # Loop over all motor tick records.
    # This is the particle filter loop, with prediction and correction.
    f = open("particle_filter_mean.txt", "w")
    for i in range(len(logfile.motor_ticks)):
        # Prediction.
        control = [x * ticks_to_mm for x in logfile.motor_ticks[i]]
        pf.predict(control)

        # Correction.
        cylinders = get_cylinders_from_scan(logfile.scan_data[i], depth_jump,
            minimum_valid_distance, cylinder_offset)
        pf.correct(cylinders, reference_cylinders)

        # Output particles.
        pf.print_particles(f)
        
        # Output state estimated from all particles.
        mean = pf.get_mean()
        print("F %.0f %.0f %.3f" %\
              (mean[0] + scanner_displacement * cos(mean[2]),
               mean[1] + scanner_displacement * sin(mean[2]),
               mean[2]), file=f)

    f.close()

### Check the output using the logfile viewer.
Since the particle filter uses random draws in the prediction and correction steps, you may re-run the cell above to get a different result! Usually, it will have bumps in different locations. The result is discussed in the next video below.

In [27]:
# Execute this to run the interactive viewer.
import ipy_logfile_viewer as lfv
v = lfv.IPYLogfileViewer(files=["particle_filter_mean.txt", "robot4_reference.txt"])

## Resampling, initialization, and error ellipses

In [28]:
# If you don't see a video below, run this cell.
from IPython.display import IFrame
IFrame("https://www.youtube.com/embed/Il2xzNchp1M" if "YouTube" in globals() else "//av.tib.eu/player/49023",
       width=560, height=315)

### Run this code to produce the error ellipses.
Note that if you use a large number of particles, this may take a while (in a Jupyter Notebook, the *kernel status indicator* will stay *busy* for a while).

In [29]:
# The particle filter, prediciton and correction.
# In addition to the previous code:
# 1.
# the second moments are computed and are output as an error ellipse and
# heading variance.
# 2.
# the particles are initialized uniformly distributed in the arena, and a
# larger number of particles is used.
# 3.
# predict and correct are only called when control is nonzero.
#
# slam_08_d_density_error_ellipse.
from lego_robot import *
from math import sin, cos, pi
import random
import numpy as np

class ParticleFilter_7(ParticleFilter_6):
    # *** Modification 1: Extension: This computes the error ellipse.
    def get_error_ellipse_and_heading_variance(self, mean):
        """Returns a tuple: (angle, stddev1, stddev2, heading-stddev) which is
           the orientation of the xy error ellipse, the half axis 1, half axis 2,
           and the standard deviation of the heading."""
        center_x, center_y, center_heading = mean
        n = len(self.particles)
        if n < 2:
            return (0.0, 0.0, 0.0, 0.0)

        # Compute covariance matrix in xy.
        sxx, sxy, syy = 0.0, 0.0, 0.0
        for p in self.particles:
            dx = p[0] - center_x
            dy = p[1] - center_y
            sxx += dx * dx
            sxy += dx * dy
            syy += dy * dy
        cov_xy = np.array([[sxx, sxy], [sxy, syy]]) / (n-1)

        # Get variance of heading.
        var_heading = 0.0
        for p in self.particles:
            dh = (p[2] - center_heading + pi) % (2*pi) - pi
            var_heading += dh * dh
        var_heading = var_heading / (n-1)

        # Convert xy to error ellipse.
        eigenvals, eigenvects = np.linalg.eig(cov_xy)
        ellipse_angle = atan2(eigenvects[1,0], eigenvects[0,0])

        return (ellipse_angle, sqrt(abs(eigenvals[0])),
                sqrt(abs(eigenvals[1])),
                sqrt(var_heading))

if __name__ == '__main__':
    # Robot constants.
    scanner_displacement = 30.0
    ticks_to_mm = 0.349
    robot_width = 155.0

    # Cylinder extraction and matching constants.
    minimum_valid_distance = 20.0
    depth_jump = 100.0
    cylinder_offset = 90.0

    # Filter constants.
    control_motion_factor = 0.35  # Error in motor control.
    control_turn_factor = 0.6  # Additional error due to slip when turning.
    measurement_distance_stddev = 200.0  # Distance measurement error of cylinders.
    measurement_angle_stddev = 15.0 / 180.0 * pi  # Angle measurement error.

    # Generate initial particles. Each particle is (x, y, theta).
    # *** Modification 2: Generate the particles uniformly distributed.
    # *** Also, use a large number of particles.
    # Change the following value "30" to something larger, e.g. 300 or 500.
    # NOTE: as the execution then takes a while, set it back to 30 before you "validate"
    # or "submit".
    number_of_particles = 30 # Set this larger. Set smaller before submission.
    # Alternative: uniform init.
    initial_particles = []
    for i in range(number_of_particles):
        initial_particles.append((
            random.uniform(0.0, 2000.0), random.uniform(0.0, 2000.0),
            random.uniform(-pi, pi)))

    # Setup filter.
    pf = ParticleFilter_7(initial_particles,
                          robot_width, scanner_displacement,
                          control_motion_factor, control_turn_factor,
                          measurement_distance_stddev,
                          measurement_angle_stddev)

    # Read data.
    logfile = LegoLogfile()
    logfile.read("robot4_motors.txt")
    logfile.read("robot4_scan.txt")
    logfile.read("robot_arena_landmarks.txt")
    reference_cylinders = [l[1:3] for l in logfile.landmarks]

    # Loop over all motor tick records.
    # This is the particle filter loop, with prediction and correction.
    f = open("particle_filter_ellipse.txt", "w")
    for i in range(len(logfile.motor_ticks)):
        control = [x * ticks_to_mm for x in logfile.motor_ticks[i]]
        # *** Modification 3: Call the predict/correct step only if there
        # *** is nonzero control.
        if control != [0.0, 0.0]:
            # Prediction.
            pf.predict(control)

            # Correction.
            cylinders = get_cylinders_from_scan(logfile.scan_data[i], depth_jump,
                minimum_valid_distance, cylinder_offset)
            pf.correct(cylinders, reference_cylinders)

        # Output particles.
        pf.print_particles(f)
        
        # Output state estimated from all particles.
        mean = pf.get_mean()
        print("F %.0f %.0f %.3f" %\
              (mean[0] + scanner_displacement * cos(mean[2]),
               mean[1] + scanner_displacement * sin(mean[2]),
               mean[2]), file=f)

        # Output error ellipse and standard deviation of heading.
        errors = pf.get_error_ellipse_and_heading_variance(mean)
        print("E %.3f %.0f %.0f %.3f" % errors, file=f)

    f.close()

### Check the results using the logfile viewer.
You may occasionally see that the filter is not able to catch up with the correct position, and the resulting trajectory is "complete nonsense". If you encounter this, re-run the previous cell. If it persists, increase `number_of_particles`.

In [30]:
# Execute this to run the interactive viewer.
import ipy_logfile_viewer as lfv
v = lfv.IPYLogfileViewer(files=["particle_filter_ellipse.txt", "robot4_reference.txt"])

### Here is one last question:
In the *variance of weights* approach discussed in the previous video, when should we resample?
- If the variance of weights is small  *or*
- If the variance of weights is large?

In [31]:
# We will have to resample if the variance is... answer with either "small" or "large".
if_the_variance_is = "large"
# YOUR CODE HERE
#raise NotImplementedError()

In [32]:
from hashlib import shake_128
def public_string_test(the_answer_string, reference):
    m = shake_128()
    m.update(the_answer_string.encode())
    return m.hexdigest(4) == reference
assert public_string_test("variance="+if_the_variance_is.strip().lower(), '45eeff02'), "Oh no, your answer is wrong!"